<h1>Group By, Having & Count</h1>

<h2>Introduction</h2>

<p>Now that you can select raw data, you're ready to learn how to group your data and count things within those groups. This can help you answer questions like:</p>

<ul>
<li>How many of each kind of fruit has our store sold?</li>
<li>How many species of animal has the vet office treated?</li>
</ul>

<p>To do this, you'll learn about three new techniques: <strong>GROUP BY</strong>, <strong>HAVING</strong> and <strong>COUNT()</strong>. Once again, we'll use this made-up table of information on pets.</p>

<p><img src="https://storage.googleapis.com/kaggle-media/learn/images/fI5Pvvp.png" alt=""></p>


<h2>COUNT()</h2>

<p><strong>COUNT()</strong>, as you may have guessed from the name, returns a count of things. If you pass it the name of a column, it will return the number of entries in that column.</p>

<p>For instance, if we <strong>SELECT</strong> the <strong>COUNT()</strong> of the <code>ID</code> column in the <code>pets</code> table, it will return 4, because there are 4 ID's in the table.</p>

<p><img src="https://storage.googleapis.com/kaggle-media/learn/images/Eu5HkXq.png" alt=""></p>

<p><strong>COUNT()</strong> is an example of an <strong>aggregate function</strong>, which takes many values and returns one. (Other examples of aggregate functions include <strong>SUM()</strong>, <strong>AVG()</strong>, <strong>MIN()</strong>, and <strong>MAX()</strong>.)  As you'll notice in the picture above, aggregate functions introduce strange column names (like <code>f0__</code>).  Later in this tutorial, you'll learn how to change the name to something more descriptive.</p>

<h2>GROUP BY</h2>

<p><strong>GROUP BY</strong> takes the name of one or more columns, and treats all rows with the same value in that column as a single group when you apply aggregate functions like <strong>COUNT()</strong>.</p>
<p>For example, say we want to know how many of each type of animal we have in the <code>pets</code> table. We can use <strong>GROUP BY</strong> to group together rows that have the same value in the <code>Animal</code> column, while using <strong>COUNT()</strong> to find out how many ID's we have in each group.</p>

<p><img src="https://storage.googleapis.com/kaggle-media/learn/images/tqE9Eh8.png" alt=""></p>

<p>It returns a table with three rows (one for each distinct animal).  We can see that the <code>pets</code> table contains 1 rabbit, 1 dog, and 2 cats.</p>

<h2>GROUP BY ... HAVING</h2>

<p><strong>HAVING</strong> is used in combination with <strong>GROUP BY</strong> to ignore groups that don't meet certain criteria.</p>
<p>So this query, for example, will only include groups that have more than one ID in them.</p>

<p><img src="https://storage.googleapis.com/kaggle-media/learn/images/2ImXfHQ.png" alt=""></p>

<p>Since only one group meets the specified criterion, the query will return a table with only one row.</p>

<h2>Example: Which Hacker News comments generated the most discussion?</h2>

<p>Ready to see an example on a real dataset? The Hacker News dataset contains information on stories and comments from the Hacker News social networking site.</p>

<p>We'll work with the <code>full</code> table and begin by printing the first few rows.</p>

In [6]:
from google.cloud import bigquery

# Project ID
project_id = "burnished-road-363918"

# Create a "Client" object
client = bigquery.Client(project=project_id)

# Authenticate
from google.colab import auth
auth.authenticate_user()

# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "full" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,None,True,Adoum_Tech,2,1713995025,2024-04-24 21:43:45+00:00,story,40150086,<NA>,<NA>,<NA>,<NA>
1,None,None,None,True,belter,2,1713995286,2024-04-24 21:48:06+00:00,story,40150135,<NA>,<NA>,<NA>,<NA>
2,None,None,None,True,Rinzler89,1,1713995678,2024-04-24 21:54:38+00:00,story,40150207,<NA>,<NA>,<NA>,<NA>
3,None,None,None,True,stockstobuynow,1,1713995704,2024-04-24 21:55:04+00:00,story,40150212,<NA>,<NA>,<NA>,<NA>
4,None,None,None,True,FLMAN407,1,1713995772,2024-04-24 21:56:12+00:00,story,40150229,<NA>,<NA>,<NA>,<NA>


<p>Let's use the table to see which comments generated the most replies.  Since:</p>

<ul>
<li>the <code>parent</code> column indicates the comment that was replied to, and </li>
<li>the <code>id</code> column has the unique ID used to identify each comment, </li>
</ul>

<p>we can <strong>GROUP BY</strong> the <code>parent</code> column and <strong>COUNT()</strong> the <code>id</code> column in order to figure out the number of comments that were made as responses to a specific comment.  (<em>This might not make sense immediately -- take your time here to ensure that everything is clear!</em>)</p>

<p>Furthermore, since we're only interested in popular comments, we'll look at comments with more than ten replies.  So, we'll only return groups <strong>HAVING</strong> more than ten ID's.</p>

In [3]:
# Query to select comments that received more than 10 replies
query_popular = """
                SELECT parent, COUNT(id)
                FROM `bigquery-public-data.hacker_news.full`
                GROUP BY parent
                HAVING COUNT(id) > 10
                """

<p>Now that our query is ready, let's run it and store the results in a pandas DataFrame:</p>

In [7]:
# Set up the query (cancel the query if it would use too much of
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_popular, job_config=safe_config, project=project_id)

# API request - run the query, and convert the results to a pandas DataFrame
popular_comments = query_job.to_dataframe()

# Print the first five rows of the DataFrame
popular_comments.head()

,parent,f0_
0,17022963,140
1,17075261,78
2,17090776,49
3,17136052,40
4,17151046,41


<p>Each row in the <code>popular_comments</code> DataFrame corresponds to a comment that received more than ten replies.  For instance, the comment with ID <code>801208</code> received <code>56</code> replies.</p>


<h2>Aliasing and other improvements</h2>

<p>A couple hints to make your queries even better:</p>

<ul>
<li>The column resulting from <code>COUNT(id)</code> was called <code>f0__</code>. That's not a very descriptive name. You can change the name by adding <code>AS NumPosts</code> after you specify the aggregation. This is called <strong>aliasing</strong>, and it will be covered in more detail in an upcoming lesson.</li>
<li>If you are ever unsure what to put inside the <strong>COUNT()</strong> function, you can do <code>COUNT(1)</code> to count the rows in each group. Most people find it especially readable, because we know it's not focusing on other columns. It also scans less data than if supplied column names (making it faster and using less of your data access quota).</li>
</ul>

<p>Using these tricks, we can rewrite our query:</p>

In [8]:
# Improved version of earlier query, now with aliasing & improved readability
query_improved = """
                 SELECT parent, COUNT(1) AS NumPosts
                 FROM `bigquery-public-data.hacker_news.full`
                 GROUP BY parent
                 HAVING COUNT(1) > 10
                 """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_improved, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
improved_df = query_job.to_dataframe()

# Print the first five rows of the DataFrame
improved_df.head()

,parent,NumPosts
0,1703197,66
1,1706908,58
2,2051288,69
3,20330818,89
4,20402070,59


<p>Now you have the data you want, and it has descriptive names. That's good style.</p>


<h2>Note on using <strong>GROUP BY</strong></h2>

<p>Note that because it tells SQL how to apply aggregate functions (like <strong>COUNT()</strong>), it doesn't make sense to use <strong>GROUP BY</strong> without an aggregate function.  Similarly, if you have any <strong>GROUP BY</strong> clause, then all variables must be passed to either a</p>

<ol>
<li><strong>GROUP BY</strong> command, or</li>
<li>an aggregation function.</li>
</ol>

<p>Consider the query below:</p>

In [9]:
query_good = """
             SELECT parent, COUNT(id)
             FROM `bigquery-public-data.hacker_news.full`
             GROUP BY parent
             """

<p>Note that there are two variables: <code>parent</code> and <code>id</code>.</p>

<ul>
<li><code>parent</code> was passed to a <strong>GROUP BY</strong> command (in <code>GROUP BY parent</code>), and </li>
<li><code>id</code> was passed to an aggregate function (in <code>COUNT(id)</code>).</li>
</ul>

<p>And this query won't work, because the <code>author</code> column isn't passed to an aggregate function or a <strong>GROUP BY</strong> clause:</p>

In [10]:
query_bad = """
            SELECT `by` AS author, parent, COUNT(id)
            FROM `bigquery-public-data.hacker_news.full`
            GROUP BY parent
            """

<p>If make this error, you'll get the error message <code>SELECT list expression references column (column's name) which is neither grouped nor aggregated at</code>.</p>

<p>You may notice the <code>`by`</code> column in this query is surrounded by backticks. This is because <strong>BY</strong> is a reserved keyword used in clauses including <strong>GROUP BY</strong>. In BigQuery reserved keywords used as identifiers must be quoted in backticks to avoid an error. We also make subsequent references to this column more readable by adding an alias to rename it to <code>author</code>.</p>